# PhySO-RANS

## Packages import

In [ ]:
# External packages
import os
import torch
import numpy as np

# Internal code import
import physo
from physo.learn import monitoring
from physo.task  import benchmark

In [ ]:
# Device
DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'
print(DEVICE)

In [ ]:
# Seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

## Test case

#### Data points

In [ ]:
DIM = 2

Retau = 500. 
H = 1. 
Ub = 0.028
nu = 5e-06
Re = 5600
rho = Re * nu / (H * Ub)

In [ ]:
# Shape of the hill
def profile(yy, a):
    y = np.array(yy)
    x = y * 28
    ya = y

    h = np.zeros(len(x))

    for i in range(len(x)):
        if (x[i]>=0) and (x[i]<54) :
            ya[i] *= a
        elif (x[i]>54) and (x[i]<=126) :
            ya[i] -= (54/28.0*(1-a))
        elif (x[i]>126) and (x[i]<=198) :
            ya[i] -= (54/28.0*(1-a))
        elif (x[i]>198) and (x[i]<=252) :
            ya[i] -= (54/28.0*(1-a))
            ya[i] -= (x[i]-198)*(1-a)/28.0
    
    for i in range(len(x)):
        if x[i] > 126.0 :
            x[i] = 252.0 - x[i]

        if (x[i]>=0) and (x[i]<9) :
            h[i] = np.minimum(28., 2.8e+01 + 0.0e+00*x[i] + \
                              6.775070969851e-03*x[i]**2 - 2.124527775800e-03*x[i]**3)
        elif (x[i]>=9) and (x[i]<14) :
            h[i] = 2.507355893131E+01 + 9.754803562315E-01*x[i] - \
                   1.016116352781E-01*x[i]**2 + 1.889794677828E-03*x[i]**3
        elif (x[i]>=14) and (x[i]<20) :
            h[i] = 2.579601052357E+01 + 8.206693007457E-01*x[i] - \
                   9.055370274339E-02*x[i]**2 + 1.626510569859E-03*x[i]**3
        elif (x[i]>=20) and (x[i]<30) :
            h[i] = 4.046435022819E+01 - 1.379581654948E+00*x[i] + \
                   1.945884504128E-02*x[i]**2 - 2.070318932190E-04*x[i]**3
        elif (x[i]>=30) and (x[i]<40) :
            h[i] = 1.792461334664E+01 + 8.743920332081E-01*x[i] - \
                   5.567361123058E-02*x[i]**2 + 6.277731764683E-04*x[i]**3
        elif (x[i]>=40) and (x[i]<=54) :
            h[i] = np.maximum(0., 5.639011190988E+01 - 2.010520359035E+00*x[i] + \
                              1.644919857549E-02*x[i]**2 + 2.674976141766E-05*x[i]**3)
        elif (x[i]>54) and (x[i]<=126) :
            h[i] = 0

    hout = h/28.0
    return ya, hout

In [ ]:
# Parameters for the shape of periodic hill
alpha = 0.8
yy = np.arange(0, 9, 0.01)

# Load hill shape
x_p = np.genfromtxt("./dataset/coord.csv", delimiter=',', skip_header=1)[:, 0]
y_p = np.genfromtxt("./dataset/coord.csv", delimiter=',', skip_header=1)[:, 1]


def wall_distance(x, y, yy = np.arange(0, 9, 0.01), alpha = 1.):
    [ya, hout] = profile(yy, alpha)
    distance = np.zeros_like(x)
    for i in range(len(x)):
        distance[i] = np.min(np.sqrt((x[i]-ya)**2 + (y[i]-hout)**2))
    return distance

# Calculate the wall distance
d = wall_distance(x_p, y_p, yy, alpha)

In [ ]:
# Index for four independent  Reynolds stress tensor
index = [0, 1, 4, 8]

DATA_LENGTH = len(x_p)

# Load the mean strain rate and rotation rate data
S = np.zeros((DATA_LENGTH, 3, 3))
R = np.zeros((DATA_LENGTH, 3, 3))

for i in range(3):
    for j in range(3):
        if i == j == 0:
            S[:, i, j] = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 0]
            R[:, i, j] = 0
        elif i == j == 1:
            S[:, i, j] = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 4]
            R[:, i, j] = 0
        elif i == j == 2:
            S[:, i, j] = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 8]
            R[:, i, j] = 0
        elif i == 0 and j == 1 or i == 1 and j == 0:
            S[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 1]
                                + np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 3])
            R[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 1]
                                - np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 3])
        elif i == 0 and j == 2 or i == 2 and j == 0:
            S[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 2]
                                + np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 6])
            R[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 2]
                                - np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 6])
        elif i == 1 and j == 2 or i == 2 and j == 1:
            S[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 5]
                                + np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 7])
            R[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 5]
                                - np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 7])
                        

# Load the mean flow data
U = np.genfromtxt("./dataset/U.csv", delimiter=',', skip_header=1)[: , :3]
k = np.genfromtxt("./dataset/k.csv", delimiter=',', skip_header=1)
epsilon = np.genfromtxt("./dataset/epsilon.csv", delimiter=',', skip_header=1)
nut = np.genfromtxt("./dataset/nut.csv", delimiter=',', skip_header=1)
p_gradient = np.genfromtxt("./dataset/grad(p).csv", delimiter=',', skip_header=1)[: , :3]
k_gradient = np.genfromtxt("./dataset/grad(k).csv", delimiter=',', skip_header=1)[: , :3]
U_gradient = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , :9].reshape(-1, 3, 3)
Q_criteria = np.genfromtxt("./dataset/Q.csv", delimiter=',', skip_header=1)


# Read the Reynolds stress tensor data from the DNS dataset
RS = np.genfromtxt("./dataset/tauDNS.csv", delimiter=',', skip_header=1).reshape(-1, 3, 3)


# Calculate mean flow features
q1 = Q_criteria
q2 = k
q3 = np.zeros(DATA_LENGTH)
for i in range(DATA_LENGTH):
    if np.sqrt(k[i])*d[i]/(50*nu) < 2:
        q3[i] = np.sqrt(k[i])*d[i]/(50*nu)
    else:
        q3[i] = 2
q4 = U[:, 0]*p_gradient[:, 0] + U[:, 1]*p_gradient[:, 1] + U[:, 2]*p_gradient[:, 2]
q5 = k/epsilon
q6 = np.sqrt(p_gradient[:, 0]*p_gradient[:, 0] + p_gradient[:, 1]*p_gradient[:, 1] + p_gradient[:, 2]*p_gradient[:, 2])
UiUj = np.einsum('bi,bj->bij', U, U)
q7 = np.einsum('bij,bij->b', UiUj, U_gradient)
q7 = np.abs(q7)

q8 = U[:, 0]*k_gradient[:, 0] + U[:, 1]*k_gradient[:, 1] + U[:, 2]*k_gradient[:, 2]


# Stack mean flow features
X = np.stack((q1, q2, q3, q4, q5, q6, q7, q8), axis=0)
y = (RS - 2/3 * k.reshape(-1, 1, 1) * np.eye(3) + 2 * nut.reshape(-1, 1, 1) * S).reshape(-1, 9)[:, index]

#### Sending data to device

In [ ]:
# ------ Vectors ------
# Stack of all input variables
X = torch.tensor(X).to(DEVICE)
# Output of symbolic function to guess
y = torch.tensor(y).to(DEVICE)

# ------ Constants ------
Re = torch.tensor(np.array(Re)).to(DEVICE)
nu = torch.tensor(np.array(nu)).to(DEVICE)
rho = torch.tensor(np.array(rho)).to(DEVICE)

## Run config

### Library config

In [ ]:
args_make_tokens = {
                # operations
                "op_names"             : ["add", "sub", "mul", "div", "n2",],
                "use_protected_ops"    : False,
                # input variables
                "input_var_ids"        : {"q1" : 0         , "q2": 1         , "q3": 2         , "q4": 3         , "q5": 4         , "q6": 5          , "q7": 6           , "q8": 7           , },
                "input_var_units"      : {"q1" : [0, 0, -2], "q2": [0, 2, -2], "q3": [0, 0, 0] , "q4": [0, 2, -3], "q5": [0, 0, 1] , "q6": [0, 1, -2] , "q7": [0, 2, -3]  , "q8": [0, 2, -3]  , }, # [M L T]
                "input_var_complexity" : {"q1" : 1.        , "q2": 1.        , "q3": 1.        , "q4": 1.        , "q5": 1.        , "q6": 1.         , "q7": 1.          , "q8": 1.          , },
                # free constants
                "free_constants"            : {"c"              , },
                "free_constants_init_val"   : {"c" : 1.         , },
                "free_constants_units"      : {"c" : [0, 0, 0]  , },
                "free_constants_complexity" : {"c" : 1.         , },
                    }

library_config = {"args_make_tokens"  : args_make_tokens,
                  "superparent_units" : [0, 2, -2],
                  "superparent_name"  : "y",
                }

                

### Learning config

In [ ]:
reward_config = {
                 "reward_function"     : physo.physym.reward.SquashedNRMSE, # PHYSICALITY
                 "zero_out_unphysical" : True,
                 "zero_out_duplicates" : False,
                 "keep_lowest_complexity_duplicate" : False,
                 "parallel_mode" : False,
                 "n_cpus"        : 12,
                }

In [ ]:
BATCH_SIZE = int(1e4)
MAX_LENGTH = 15
GET_OPTIMIZER = lambda model : torch.optim.Adam(
                                    model.parameters(),                
                                    lr=1e-4, #0.001, #0.0050, #0.0005, #1,  #lr=0.0025
                                                )

In [ ]:
learning_config = {
    # Batch related
    'batch_size'       : BATCH_SIZE,
    'max_time_step'    : MAX_LENGTH,
    'n_epochs'         : 100,
    # Loss related
    'gamma_decay'      : 0.7,
    'entropy_weight'   : 0.005,
    # Reward related
    'risk_factor'      : 0.05,
    'rewards_computer' : physo.physym.reward.make_RewardsComputer (**reward_config),
    # Optimizer
    'get_optimizer'    : GET_OPTIMIZER,
    'observe_units'    : True,
}

### Free constant optimizer config

In [ ]:
free_const_opti_args = {
            'loss'   : "MSE",
            'method' : 'LBFGS',
            'method_args': {
                        'n_steps' : 15,
                        'tol'     : 1e-8,
                        'lbfgs_func_args' : {
                            'max_iter'       : 4,
                            'line_search_fn' : "strong_wolfe",
                                             },
                            },
        }

### Priors config

In [ ]:
priors_config  = [
                #("UniformArityPrior", None),
                # LENGTH RELATED
                ("HardLengthPrior"  , {"min_length": 4, "max_length": MAX_LENGTH, }),
                ("SoftLengthPrior"  , {"length_loc": 8, "scale": 5, }),
                # RELATIONSHIPS RELATED
                ("NoUselessInversePrior"  , None),
                ("PhysicalUnitsPrior", {"prob_eps": np.finfo(np.float32).eps}), # PHYSICALITY
                # ("NestedFunctions", {"functions":["exp", ], "max_nesting" : 1}),
                # ("NestedFunctions", {"functions":["log",], "max_nesting" : 1}),
                # ("NestedTrigonometryPrior", {"max_nesting" : 1}),
                # ("OccurrencesPrior", {"targets" : ["nu",], "max" : [2,] }),
                 ]

### Cell config

In [ ]:
cell_config = {
    "hidden_size" : 128,
    "n_layers"    : 1,
    "is_lobotomized" : False,
}

### Logger

In [ ]:
run_logger = []
run_visualiser = []

for i in range(len(index)):
    save_path_training_curves = 'RS_curves_' + str(index[i]) + '.png'
    save_path_log             = 'RS_' + str(index[i]) + '.log'

    run_logger.append(monitoring.RunLogger(save_path = save_path_log, 
                                      do_save = False,
                                      ))

    run_visualiser.append(monitoring.RunVisualiser (epoch_refresh_rate = 5,
                                           save_path = save_path_training_curves,
                                           do_show   = True,
                                           do_prints = True,
                                           do_save   = True, 
                                           ))

### Run config

In [ ]:
run_config = []
for i in range(len(index)):
    run_config.append({
        "learning_config"      : learning_config,
        "reward_config"        : reward_config,
        "free_const_opti_args" : free_const_opti_args,
        "library_config"       : library_config,
        "priors_config"        : priors_config,
        "cell_config"          : cell_config,
        "run_logger"           : run_logger[i],
        "run_visualiser"       : run_visualiser[i],
    })

## Run

In [ ]:
for i in range(len(index)):
    benchmark.dummy_epoch(X, y[:, i], run_config[i]) # Dummy epoch for prior tuning
    rewards, candidates = physo.fit (X, y[:, i], run_config[i],
                                    stop_reward = 0.9999, 
                                    stop_after_n_epochs = 5)